# 03 Algorithms

## 3.4 Monte Carlo Methods

### Monte Carlo Methods
在Policy Iteration计算过程有两个关键步骤：1. Policy Evaluation，2. Policy Improvement。其中Policy Evaluation需要通过迭代求解Bellman Expectation Equation来实现，这个过程的计算量非常大，尤其是当状态空间很大的时候。特别地，在计算状态价值$v_{\pi_k}(s)$的过程中，是通过计算状态$s$下所有可能的动作$a$的价值期望值来实现的，即：
$$v_{\pi_k}(s) = \sum_{a \in A} \pi(a|s) \sum_{s' \in S} p(s'|s, a)[r(s, a, s') + \gamma v_{\pi_k}(s')]$$
这个公式的计算量非常大，尤其是当状态空间很大的时候。

Mente Carlo Methods从动作价值的定义出发（动作价值是从当前状态出发能够获得的期望回报），通过采样大量的episode来估计$v_{\pi_k}(s)$的值，从而避免了复杂的矩阵运算，称为**Model-free近似**：
$$
\begin{aligned}
q_{\pi_k} (s, a) &= \mathbb{E} \left[ G_t | S_t = s, A_t = a \right] \\
&= \mathbb{E} \left[ R_{t+1} + \gamma G_{t+1} | S_t = s, A_t = a \right] \\
&= \mathbb{E} \left[ R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \cdots | S_t = s, A_t = a \right] \\
&\approx \frac{1}{N} \sum_{i=1}^N g_{\pi_k}^i(s, a) \\
\end{aligned}
$$

其中，$g_{\pi_k}^i(s, a)$是从状态$s$出发，经过动作$a$后，按照策略$\pi_k$所得到的总回报，$N$是采样的episode数量。

具体来说，在Policy Evaluation的过程中，我们只需要知道当前策略$\pi$下从状态$s$出发，经历一系列的状态转移到达终止状态所得到的回报之和即可。这个过程不需要考虑状态之间的转移概率和即时奖励，只需要采样足够多的episode，然后计算每个状态的平均回报值即可。

**动作价值计算流程:**
1. 使用第$k$步的策略$\pi_k$采样若干条轨迹（episodes），每条轨迹包含多个状态和动作：
   $$s_0^{(i)} \xrightarrow{a_0^{(i)}} s_1^{(i)} \xrightarrow{a_1^{(i)}} s_2^{(i)} \xrightarrow{a_2^{(i)}} \dots s_{T-1}^{(i)} \xrightarrow{a_{T-1}^{(i)}} s_T^{(i)}$$

    $$s_0^{(i)} \xrightarrow{a_0^{(i)}} r_0^{(i)}, s_1^{(i)} \xrightarrow{a_1^{(i)}} r_1^{(i)}, s_2^{(i)} \xrightarrow{a_2^{(i)}} r_2^{(i)}, \dots s_{T-1}^{(i)} \xrightarrow{a_{T-1}^{(i)}} r_{T-1}^{(i)}, s_T^{(i)}$$
    其中$s_t^{(i)}$表示第$i$条轨迹在第$t$步的状态，$a_t^{(i)}$表示第$i$条轨迹在第$t$步的动作，$r_t^{(i)}$表示第$i$条轨迹在第$t$步的奖励。

2. 计算每条轨迹的回报：
   $$G_t^{(i)} = r_t^{(i)} + \gamma r_{t+1}^{(i)} + \gamma^2 r_{t+2}^{(i)} + \dots + \gamma^{T-t} r_{T}^{(i)}$$
   其中$\gamma$是折扣因子。
3. 计算每条轨迹的平均回报：
   $$\bar{G}_t = \frac{1}{N} \sum_{i=1}^N G_t^{(i)}$$
   其中$N$是轨迹的数量。
   这里通常使用增量法进行计算，即：
   $$ N(s) \leftarrow N(s) + 1 $$

   $$ \bar{G}(s) \leftarrow \bar{G}(s) + \frac{1}{N(s)} (G_t^{(i)} - \bar{G}(s)) $$
   这样可以避免存储所有的回报值，节省内存。

### Monte Carlo Basic
* 随机初始化$\pi_0$
* $for\ k = 0, 1, 2, ... \ do:$
* $\qquad$ $for \ s \in \cal S \ do:$
* $\qquad\qquad$ $for \ a \in \cal A(s) \ do:$
* $\qquad\qquad\qquad$ 遵循策略$\pi_k$，生成从$(s, a)$出发的轨迹，直到终止状态;
* $\qquad\qquad\qquad$ $Policy Evaluation:$
* $\qquad\qquad\qquad$ 对于轨迹中的每一个状态-动作对$(s, a)$，计算$G_t^{(i)}$，并更新$N(s) \leftarrow N(s) + 1$和$\bar{G}(s) \leftarrow \bar{G}(s) + \frac{1}{N(s)} (G_t^{(i)} - \bar{G}(s))$
* $\qquad\qquad$ $end \ for$
* $\qquad\qquad$ $Policy \ Improvement:$
* $\qquad\qquad$ $a_k^*(s) \leftarrow \arg\max_a \bar{G}(s)$
* $\qquad\qquad$ $\pi_{k+1}(a|s) = 1 \ if \ a_k^*(s)=a, \ else \ 0$
* $\qquad\qquad$ 如果$\pi_{k+1} = \pi_k$，则停止; 否则继续。
* $\qquad$ $end \ for$
* $end \ for$

### Monte Carlo Exploring Starts Algorithm
- 对于所有的状态-动作对，初始化策略$\pi_0(s, a)$、动作价值$q(s, a)$、回报$\bar{G}(s, a)$为0、回报计数$N(s, a)$为0
- 对于每个episode:
  - 随机选择初始状态$s_0 \sim \mu(s)$和初始动作$a_0 \sim \pi_0(s_0)$，确保所有的状态-动作对都能够被选择，其中$\mu(s)$是初始状态分布。
  - 生成一个episode $s_0, a_0, r_1, s_1, a_1, ..., s_{T-1}, a_{T-1}, r_T$，根据策略$\pi_k$和环境模型。
  - 初始化每一个episode的回报$g_{\pi_k}^i (s, a) \leftarrow 0$。
  - $for \ t = T-1 \ down \ to \ 0$：
  - $\qquad$ $g_{\pi_k}^i (s, a) \leftarrow r_{t+1} + \gamma g_{\pi_k}^i (s_{t+1}, a_{t+1})$
  - $\qquad$ $\bar{G}(s, a) \leftarrow \bar{G}(s, a) + g_{\pi_k}^i (s, a)$
  - $\qquad$ $N(s, a) \leftarrow N(s, a) + 1$
  - $\qquad$ Policy Evaluation: $q_{\pi_k}(s, a) \leftarrow \bar{G}(s, a) / N(s, a)$
  - $\qquad$ Policy Improvement: $\pi_{k+1} (s) \leftarrow argmax_{a} q_{\pi_k}(s, a)$
  - $\qquad$ $if \ \pi_{k+1} (s) = \pi_k(s) \ for \ all \ s$:  $break$

### Example

In [1]:
import time

import numpy as np
import gymnasium as gym

In [2]:
class MonteCarloExploringStarts:
    """ Monte Carlo Exploring Starts Algorithm for Frozen-Lake environment """

    def __init__(self, env, gamma=0.9, num_episodes=1000, epsilon=0.1, epsilon_decay=0.99):
        self.env = env
        self.gamma = gamma
        self.num_episodes = num_episodes
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay

        self.policy = np.ones((env.observation_space.n, env.action_space.n)) / env.action_space.n
        self.Q = np.zeros((env.observation_space.n, env.action_space.n))
        self.returns = np.zeros((env.observation_space.n, env.action_space.n))
        self.counts = np.zeros((env.observation_space.n, env.action_space.n))

    def select_action(self, state):
        if np.random.rand() < self.epsilon:
            return self.env.action_space.sample()
        else:
            return np.argmax(self.Q[state])

    @staticmethod
    def custom_reward(done, reward):
        if done and reward == 1:
            return 10
        elif done and reward == 0:
            return -5
        else:
            return -0.1

    def generate_episode(self, state):
        episode = []
        done = False
        while not done:
            action = self.select_action(state)
            trans = self.env.unwrapped.P[state][action]
            idx = np.random.choice(range(len(trans)))
            prob, next_state, reward, done = trans[idx]

            reward = self.custom_reward(done, reward)
            episode.append((state, action, reward))
            state = next_state
        return episode

    def update(self, episode):
        G = 0
        visited_state_actions = set()
        for state, action, reward in reversed(episode):
            G = self.gamma * G + reward
            if (state, action) not in visited_state_actions:
                self.returns[state, action] += G
                self.counts[state, action] += 1
                self.Q[state, action] = self.returns[state, action] / self.counts[state, action]
                best_action = np.argmax(self.Q[state])
                self.policy[state] = np.zeros_like(self.Q[state])
                self.policy[state, best_action] = 1
                visited_state_actions.add((state, action))

    def run(self):
        for _ in range(self.num_episodes):
            state = self.env.observation_space.sample()
            episode = self.generate_episode(state)
            self.update(episode)
            self.epsilon *= self.epsilon_decay
            self.epsilon = max(self.epsilon, 0.1)

    def visualize_policy(self, delay=0.5):
        state, info = self.env.reset()
        done = False

        while not done:
            self.env.render()
            action = np.argmax(self.policy[state])
            state, reward, terminated, truncated, info = self.env.step(action)
            done = terminated or truncated
            time.sleep(delay)

        self.env.render()
        self.env.close()


In [3]:
environment = gym.make('FrozenLake-v1', desc=None, map_name='8x8', is_slippery=True, render_mode='human')
environment.reset()

2025-02-13 17:32:33.699 python[71245:45435508] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-13 17:32:33.699 python[71245:45435508] +[IMKInputSession subclass]: chose IMKInputSession_Modern


(0, {'prob': 1})

In [4]:
agent = MonteCarloExploringStarts(environment, gamma=0.9, num_episodes=5000, epsilon=0.9)
agent.run()
print(f"Optimal Policy: {agent.policy}")

Optimal Policy: [[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]]


In [5]:
# Visualize the policy in the environment
environment.reset()
agent.visualize_policy(delay=0.005)